In [ ]:
!pip install -q transformers sentencepiece datasets accelerate evaluate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 9.6 MB/s eta 0:00:00


# **Dataset**

In [ ]:
from datasets import load_dataset

ds = load_dataset("thainq107/iwslt2015-en-vi")

README.md:   0%|          | 0.00/522 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/181k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/133317 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1268 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1268 [00:00<?, ? examples/s]

# **Tokenizer**

In [ ]:
from transformers import AutoTokenizer

model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

# **Encoding**

In [ ]:
import torch

MAX_LEN = 75

def preprocess_function(sentences):
    # Biến các câu từ thành token (IDs)
    input_ids = tokenizer(
        sentences["en"], padding="max_length", truncation=True, max_length=MAX_LEN
    )["input_ids"]

    labels = tokenizer(
        sentences["vi"], padding="max_length", truncation=True, max_length=MAX_LEN
    )["input_ids"]

    # Chúng ta muốn model khi finetuning sẽ không quan tâm đến vị trí padding (<pad>)
    # Hugging Face có quy ước: nếu một vị trí là -100 => bỏ qua
    labels = [
        [-100 if item == tokenizer.pad_token_id else item for item in label]
        for label in labels
    ]

    return {
        "input_ids": torch.tensor(input_ids),
        "labels": torch.tensor(labels)
    }

In [ ]:
preprocess_ds = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/133317 [00:00<?, ? examples/s]

Map:   0%|          | 0/1268 [00:00<?, ? examples/s]

Map:   0%|          | 0/1268 [00:00<?, ? examples/s]

# **Model**

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

# **Evaluate**

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("sacrebleu")

def preprocess_text(preds, label):
    # Loại bỏ khoảng trắng thừa
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels] # Theo yêu cầu Sacrebleu: CTDL List[List[str]]

    return preds, labels

In [ ]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    # Thay tất cả các value = -100 thành 1
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)

    # Chuyển toàn bộ token IDs => văn bản
    decoded_preds = tokenizer.batch_decode(
        preds, skip_special_tokens=True, clean_up_tokenization_spaces=True
    )

    # Thay tất cả các value = -100 thành 1
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Chuyển toàn bộ token IDs => văn bản
    decoded_labels = tokenizer.batch_decode(
        labels, skip_special_tokens=True, clean_up_tokenization_spaces=True
    )

    decoded_preds, decoded_labels = preprocess_text(decoded_preds, decoded_labels)

    # Tính điểm BLEU
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    return result

# **Finetuning**

In [ ]:
from transformers import Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./en-vi-mbart50",
    logging_dir="logs",
    logging_steps=1000,
    predict_with_generate=True,
    eval_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    save_total_limit=1,
    num_train_epochs=3,
    load_best_model_at_end=True,
    # report_to="wandb"
)

# Gộp data thành các batch để training theo batch
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model
)

trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=preprocess_ds['train'],
    eval_dataset=preprocess_ds['validation'],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cybersoft-codingcamp (cybersoft-codingcamp-cybersoft-academy-o-t-o-chuy-n-gia-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

# **Inference**

In [ ]:
model_name = "cybersoft123/en-vi-mbart50"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/992 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

In [ ]:
src_text = "In the next step, we consider the next possible tokens for each of the three branches we created in the previous step."
encoded_text = tokenizer(src_text, return_tensors="pt")
generated_tokens = model.generate(
    **encoded_text,
    num_beams=5,
)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

['Trong bước tiếp theo , chúng tôi xem xét các token tiếp theo cho mỗi trong ba nhánh mà chúng tôi tạo ra trong bước trước .']